In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import os

from battleship.board import Board
from analysis import load_dataset
from battleship.utils import PROJECT_ROOT

In [ ]:
HUMAN_EXPERIMENT_NAME = "battleship-final-data"
PATH_DATA = os.path.join("data", HUMAN_EXPERIMENT_NAME)

df = load_dataset(PATH_DATA, use_gold=True)

In [ ]:
df.columns.unique()

In [ ]:
def retrieve_example(question_text: str):
    result = df[df["messageText"].str.contains(question_text)]
    if len(result) != 1:
        raise ValueError("Expected exactly one matching message")
    result = result.iloc[0]

    answer_index = result.name + 1
    print(answer_index)
    answer = df.loc[answer_index]

    print(result.gameID)
    print(result.roundID)
    print(question_text)
    print(result.board_id)

    moves_remaining = 40 - (result["hits"] + result["misses"])
    questions_remaining = int(result["questionsRemaining"])
    print(f"Moves remaining: {moves_remaining}, Questions remaining: {questions_remaining}")

    print(f"Answer: {answer.messageText}")
    print(f"Gold Answer: {answer.gold_answer}")

    print(f"stateful: {answer.gold_stateful}")
    print(f"discourse: {answer.gold_discourse}")
    print(f"vague: {answer.gold_vague}")
    print(f"ambiguous: {answer.gold_ambiguous}")
    print(f"unanswerable: {answer.gold_unanswerable}")

    true_board = Board.from_trial_id(result.board_id)
    partial_board = Board.from_occ_tiles(result.occTiles)

    print(true_board.ship_tracker(partial_board))

    spotter_view = true_board.spotter_view(partial_board)
    # display(spotter_view)

    combined_view = true_board.combined_view(partial_board, questions_remaining=questions_remaining, moves_remaining=moves_remaining)

    import matplotlib.pyplot as plt

    caption = f"gameID: {result.gameID}, roundID: {result.roundID}"
    combined_view.suptitle(caption, fontsize=12, y=0.05)

    OUTPUT_DIR = os.path.join(PROJECT_ROOT, "..", "battleship-iclr-2026", "iclr2026/figures/appendix_examples")
    print(OUTPUT_DIR)
    assert os.path.exists(OUTPUT_DIR), f"Output directory {OUTPUT_DIR} does not exist"
    combined_view.savefig(os.path.join(OUTPUT_DIR, f"example_{result.gameID}_{result.roundID}.pdf"), dpi=300, bbox_inches="tight")

In [ ]:
df[df["messageType"] == "question"].sample(n=5, random_state=42)["messageText"]

In [ ]:
combined_indices = list(question_rows.index) + list(answer_rows.index)
combined_indices

In [ ]:
answer_rows = df[df.gold_unanswerable].sample(n=5, random_state=42)
question_rows = df.loc[answer_rows.index - 1]

combined_indices = sorted(list(question_rows.index) + list(answer_rows.index))

df.loc[combined_indices][
    [
        "messageText",
        "gold_stateful",
        "gold_discourse",
        "gold_vague",
        "gold_ambiguous",
        "gold_unanswerable",
    ]
]

In [ ]:
df.loc[9889]["messageText"]

In [ ]:
question_text = "Does E7 have a part of a ship?"
retrieve_example(question_text)

In [ ]:
question_text = "any parts of an undiscovered boat in Row e?"
retrieve_example(question_text)

In [ ]:
question_text = "Is orange verticle?"
retrieve_example(question_text)

In [ ]:
question_text = "any ship touching that miss?"
retrieve_example(question_text)

In [ ]:
question_text = "Does it touch E4 or C4?"
retrieve_example(question_text)

In [ ]:
question_text = "Is red close to green?"
retrieve_example(question_text)

In [ ]:
question_text = "is one on f after 5 or on g"
retrieve_example(question_text)

In [ ]:
question_text = "where are the ships?"
retrieve_example(question_text)

In [ ]:
with pd.option_context('display.max_colwidth', None, 'display.max_rows', None):
    display(df[(df["messageType"] == "question") & (df["messageText"].str.contains("lol"))]["messageText"])

In [ ]:
question_text = "you lied.... Is it on row b?"
retrieve_example(question_text)

In [ ]:
question_text = "We killin it bro lets do this! LETS DO THIS! lol. Alright A - C , 1"
retrieve_example(question_text)